In [15]:
import sys
sys.path.append("/n/home12/binxuwang/Github/mini_edm")
sys.path.append("/n/home12/binxuwang/Github/DiffusionReasoning")
sys.path.append("/n/home12/binxuwang/Github/DiT")
import time
import os
from os.path import join
import pickle as pkl
import torch
import torch as th
from tqdm.auto import tqdm
import numpy as np
import seaborn as sns
import einops
import matplotlib.pyplot as plt
from collections import defaultdict
from easydict import EasyDict as edict
import matplotlib.pyplot as plt 
plt.rcParams['figure.dpi'] = 72
plt.rcParams['figure.figsize'] = [6.0, 4.0]
plt.rcParams['figure.edgecolor'] = (1, 1, 1, 0)
plt.rcParams['figure.facecolor'] = (1, 1, 1, 0)
# vector graphics type
plt.rcParams['pdf.fonttype'] = 42
plt.rcParams['ps.fonttype'] = 42
from train_edm import create_model, edm_sampler, EDM
from edm_utils import edm_sampler_inpaint, create_edm, get_default_config, create_edm_new, edm_sampler_stoch
from dataset_utils import train_data2attr_tsr,load_raw_data,load_PGM_abstract
from rule_new_utils import check_r3_r2_batch, infer_rule_from_sample_batch, compute_rule_statistics
import circuit_toolkit
from circuit_toolkit.layer_hook_utils import print_specific_layer, get_module_name_shapes, featureFetcher_module


In [5]:
!ls {exproot}
!rm -r {exproot}/090*

090-RAVEN10_abstract-uncond-DiT_S_1-stream0_16M_heldout0_20240711-0204
BaseBlnr_RAVEN10_abstract_20240212-2142
BaseBlnr_RAVEN10_abstract_onehot_20240212-2143
BaseBlnrX3_new_RAVEN10_abstract_20240313-1736
BaseBlnrX3_new_RAVEN10_abstract_onehot_20240313-1736
BaseBlnrX3_new_stream0_16M_heldout0_RAVEN10_abstract_20240921-2218
BaseBlnrX3_new_stream0_16M_RAVEN10_abstract_20240921-2218
base_cifar10_20240130-2317
base_cifar10_20240130-2318
base_gabor_prime_20240130-2251
base_gabor_sf_20240130-2306
base_mnist_20240129-1342
base_mnist_20240129-1406
base_mnist_20240130-2207
base_RAVEN10_20240131-2049
base_RAVEN10_abstract_20240201-0036
Base_RAVEN10_abstract_20240212-2139
base_RAVEN10_abstract_onehot_20240201-0147
Base_RAVEN10_abstract_onehot_20240212-2140
BBigBlnrX3_new_RAVEN10_abstract_20240313-1809
BBigBlnrX3_new_RAVEN10_abstract_onehot_20240313-1810
BBigBlnrX3_new_RAVEN10_abstract_onehot_20240314-0112
BBigBlnrX3_RAVEN10_abstract_20240305-2341
BigBlnrlrsm_RAVEN10_abstract_onehot_20240209-1411
B

In [7]:

config_mapping = {
    "BaseBlnrX3" : dict(layers_per_block=1, model_channels=64, channel_mult=[1, 2, 4], attn_resolutions=[9, 3], spatial_matching="bilinear"),
    "WideBlnrX3" : dict(layers_per_block=2, model_channels=128, channel_mult=[1, 2, 4], attn_resolutions=[9, 3], spatial_matching="bilinear"),
    "BigBlnrX3"  : dict(layers_per_block=3, model_channels=192, channel_mult=[1, 2, 4], attn_resolutions=[9, 3], spatial_matching="bilinear"),
}

import json 
import argparse

# parser = argparse.ArgumentParser()
# parser.add_argument("--expname", type=str, default="090-RAVEN10_abstract-uncond-DiT_S_1-stream0_16M_heldout0_20240711-0204")
# parser.add_argument("--epoch", type=int, default=999999)
# parser.add_argument("--batch_size", type=int, default=2048)
# parser.add_argument("--reps", type=int, default=10)
# args = parser.parse_args()
expname = r"WideBlnrX3_new_stream0_16M_heldout0_RAVEN10_abstract_20240708-2054"
epoch = 999999 #1000000
batch_size = 2048
reps = 10
# expname = args.expname
# epoch = args.epoch
# batch_size = args.batch_size
# reps = args.reps
device = "cuda"
train_steps = epoch

DATASET = "RAVEN10_abstract"
use_ema = True
exproot = r"/n/holylfs06/LABS/kempner_fellow_binxuwang/Users/binxuwang/DL_Projects/mini_edm/exps"
expdir = join(exproot, expname)
ckptdir = join(expdir, "checkpoints")
savedir = join(expdir, "uncond_samples")
os.makedirs(savedir, exist_ok=True)
model_scale = expname.split("_")[0]
config_ft = get_default_config(DATASET, **config_mapping[model_scale])
if epoch == -1:
    edm, model_EDM = create_edm_new(None, config_ft, device) 
    ckpt_str = "ckptRNDINIT"
    print("Random initialization")
else:
    ckpt_path = join(ckptdir, f"ema_{epoch}.pth")
    edm, model_EDM = create_edm_new(ckpt_path, config_ft, device) 
    ckpt_str = f"ckpt{epoch:07d}EMA"
    print(f"Loaded {ckpt_str}: from {ckpt_path}, use_ema: {use_ema}")
    
dataset_Xmean = th.tensor([1.5, 2.5, 2.5]).view(1, 3, 1, 1).to("cuda")
dataset_Xstd = th.tensor([2.5, 3.5, 3.5]).view(1, 3, 1, 1).to("cuda")


Loaded ckpt0999999EMA: from /n/holylfs06/LABS/kempner_fellow_binxuwang/Users/binxuwang/DL_Projects/mini_edm/exps/WideBlnrX3_new_stream0_16M_heldout0_RAVEN10_abstract_20240708-2054/checkpoints/ema_999999.pth, use_ema: True


In [ ]:
edm.ema.to(device).eval()
edm.model.to(device).eval()

In [ ]:
import numpy as np
from tqdm.auto import tqdm, trange
@torch.no_grad()
def edm_sampler_stoch(
    net, latents, class_labels=None, randn_like=torch.randn_like,
    num_steps=18, sigma_min=0.002, sigma_max=80, rho=7,
    S_churn=0, S_min=0, S_max=float('inf'), S_noise=1,
):
    # Adjust noise levels based on what's supported by the network.
    sigma_min = max(sigma_min, net.sigma_min)
    sigma_max = min(sigma_max, net.sigma_max)

    # Time step discretization.
    step_indices = torch.arange(num_steps, dtype=torch.float64, device=latents.device)
    t_steps = (sigma_max ** (1 / rho) + step_indices / (num_steps - 1) * (sigma_min ** (1 / rho) - sigma_max ** (1 / rho))) ** rho
    t_steps = torch.cat([net.round_sigma(t_steps), torch.zeros_like(t_steps[:1])]) # t_N = 0

    # Main sampling loop.
    x_next = latents.to(torch.float64) * t_steps[0]
    for i, (t_cur, t_next) in tqdm(enumerate(zip(t_steps[:-1], t_steps[1:]))): # 0, ..., N-1
        x_cur = x_next

        # Increase noise temporarily.
        gamma = min(S_churn / num_steps, np.sqrt(2) - 1) if S_min <= t_cur <= S_max else 0
        t_hat = net.round_sigma(t_cur + gamma * t_cur)
        x_hat = x_cur + (t_hat ** 2 - t_cur ** 2).sqrt() * S_noise * randn_like(x_cur)

        # Euler step.
        denoised = net(x_hat, t_hat, class_labels).to(torch.float64)
        d_cur = (x_hat - denoised) / t_hat
        x_next = x_hat + (t_next - t_hat) * d_cur

        # Apply 2nd order correction.
        if i < num_steps - 1:
            denoised = net(x_next, t_next, class_labels).to(torch.float64)
            d_prime = (x_next - denoised) / t_next
            x_next = x_hat + (t_next - t_hat) * (0.5 * d_cur + 0.5 * d_prime)

    return x_next


In [14]:
# Now, let's do the same for the SDE DDPM sampling
for rep in range(1):
    for steps in [5, 10, 20, 50, 100, 200, 500, 1000]:
        print(f"Sampling with Heun{steps}...rep{rep}")
        noise = th.randn(batch_size, 3, 9, 9, device="cuda", generator=th.Generator(device='cuda').manual_seed(rep))
        samples = edm_sampler(edm, noise, num_steps=steps).detach()
        samples = edm_sampler_stoch(edm, noise, num_steps=steps,).detach()
        samples = ((samples.detach() * dataset_Xstd) + dataset_Xmean).cpu()
        r3_list, r2_list, rule_col = infer_rule_from_sample_batch(samples)
        C3_count, C2_count, anyvalid_count, total = compute_rule_statistics(r3_list, r2_list, rule_col, verbose=True)
        torch.save(samples, f"{savedir}/{train_steps:07d}_Heun{steps}_rep{rep}.pt")
        torch.save({'c3_list': r3_list, 'c2_list': r2_list, 'rule_col': rule_col, 
                'c3_cnt': C3_count, 'c2_cnt': C2_count, 'anyvalid_cnt': anyvalid_count, 'total': total},
                            f'{savedir}/sample_rule_eval_{train_steps}_Heun{steps}_rep{rep}.pt')


Sampling with Heun5...rep0
C3: 0/2048 (0.00), C3 + C2: 24/2048 (0.01), AnyValid: 1065/6144 (0.17)
Sampling with Heun10...rep0
C3: 607/2048 (0.30), C3 + C2: 1063/2048 (0.52), AnyValid: 3832/6144 (0.62)
Sampling with Heun20...rep0
C3: 903/2048 (0.44), C3 + C2: 1340/2048 (0.65), AnyValid: 4385/6144 (0.71)
Sampling with Heun50...rep0
C3: 970/2048 (0.47), C3 + C2: 1409/2048 (0.69), AnyValid: 4468/6144 (0.73)
Sampling with Heun100...rep0
C3: 976/2048 (0.48), C3 + C2: 1420/2048 (0.69), AnyValid: 4489/6144 (0.73)
Sampling with Heun200...rep0
C3: 972/2048 (0.47), C3 + C2: 1421/2048 (0.69), AnyValid: 4488/6144 (0.73)
Sampling with Heun500...rep0
C3: 972/2048 (0.47), C3 + C2: 1421/2048 (0.69), AnyValid: 4486/6144 (0.73)
Sampling with Heun1000...rep0
C3: 972/2048 (0.47), C3 + C2: 1421/2048 (0.69), AnyValid: 4485/6144 (0.73)


In [18]:
stoch_kwargs = dict(S_churn=40, S_min=0.05, S_max=50, S_noise=1.003)
stoch_str = "_".join([f"{k}{v}" for k, v in stoch_kwargs.items()])
# Now, let's do the same for the SDE DDPM sampling
for rep in range(1):
    for steps in [5, 10, 20, 50, 100, 200, 500, 1000]:
        print(f"Sampling with Heun{steps}...rep{rep}")
        noise = th.randn(batch_size, 3, 9, 9, device="cuda", generator=th.Generator(device='cuda').manual_seed(rep))
        samples = edm_sampler_stoch(edm, noise, num_steps=steps,
                                    **stoch_kwargs).detach()
        samples = ((samples.detach() * dataset_Xstd) + dataset_Xmean).cpu()
        r3_list, r2_list, rule_col = infer_rule_from_sample_batch(samples)
        C3_count, C2_count, anyvalid_count, total = compute_rule_statistics(r3_list, r2_list, rule_col, verbose=True)
        torch.save(samples, f"{savedir}/{train_steps:07d}_HeunStoch{steps}_{stoch_str}_rep{rep}.pt")
        torch.save({'c3_list': r3_list, 'c2_list': r2_list, 'rule_col': rule_col, 
                'c3_cnt': C3_count, 'c2_cnt': C2_count, 'anyvalid_cnt': anyvalid_count, 'total': total},
                            f'{savedir}/sample_rule_eval_{train_steps}_HeunStoch{steps}_{stoch_str}_rep{rep}.pt')


Sampling with Heun5...rep0


0it [00:00, ?it/s]

C3: 0/2048 (0.00), C3 + C2: 35/2048 (0.02), AnyValid: 1065/6144 (0.17)
Sampling with Heun10...rep0


0it [00:00, ?it/s]

C3: 392/2048 (0.19), C3 + C2: 856/2048 (0.42), AnyValid: 3372/6144 (0.55)
Sampling with Heun20...rep0


...

0it [00:00, ?it/s]

C3: 1250/2048 (0.61), C3 + C2: 1506/2048 (0.74), AnyValid: 4719/6144 (0.77)
Sampling with Heun500...rep0


0it [00:00, ?it/s]

C3: 1274/2048 (0.62), C3 + C2: 1536/2048 (0.75), AnyValid: 4723/6144 (0.77)
Sampling with Heun1000...rep0


0it [00:00, ?it/s]

In [ ]:
edm.ema.to(device).eval()
edm.model.to(device).eval()
# Now, let's do the same for the SDE DDPM sampling
for rep in range(1):
    for steps in [5, 10, 20, 50, 100, 200, 500, 1000]:
        print(f"Sampling with Heun{steps}...rep{rep}")
        noise = th.randn(batch_size, 3, 9, 9, device="cuda", generator=th.Generator(device='cuda').manual_seed(rep))
        samples = edm_sampler_stoch(edm, noise, num_steps=steps,).detach()
        samples = ((samples.detach() * dataset_Xstd) + dataset_Xmean).cpu()
        r3_list, r2_list, rule_col = infer_rule_from_sample_batch(samples)
        C3_count, C2_count, anyvalid_count, total = compute_rule_statistics(r3_list, r2_list, rule_col, verbose=True)
        torch.save(samples, f"{savedir}/{train_steps:07d}_Heun{steps}_rep{rep}.pt")
        torch.save({'c3_list': r3_list, 'c2_list': r2_list, 'rule_col': rule_col, 
                'c3_cnt': C3_count, 'c2_cnt': C2_count, 'anyvalid_cnt': anyvalid_count, 'total': total},
                            f'{savedir}/sample_rule_eval_{train_steps}_Heun{steps}_rep{rep}.pt')
